In [17]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../scripts')  # from notebooks/ to scripts/

from trading_tax import tax
from bankroll_sizing import bankroll_sizing  


In [18]:
df = pd.read_csv(r'C:\Users\Jouke\Documents\evedata-logger\output\market_data_with_names_merged.csv', parse_dates=['date'])

In [19]:
df['date'] = pd.to_datetime(df['date'])
last_30 = df[df['date'] >= df['date'].max() - pd.Timedelta(days=30)]


In [20]:
stats = last_30.groupby(['type_id', 'type_name']).agg(
    avg_lowest=('lowest', 'mean'),
    avg_highest=('highest', 'mean'),
    avg_daily_volume=('volume', 'mean')
).reset_index()

# Calculate ROI: (sell-buy)/buy
stats['roi'] = stats.apply(
    lambda row: tax(
        row['avg_lowest'],
        row['avg_highest'],
        apply_fees=True   # Set to False if you want to ignore fees
    ),
    axis=1
)

results = bankroll_sizing(
    stats,
    bankroll=2_000_000_000,   # <-- set your current ISK bankroll
    order_pct_min=0.03,
    order_pct_max=0.05,
    min_roi=0.10,
    max_days_to_fill=3
)

filtered = stats[stats['roi'] >= 0.10]
highest_liquidity = filtered.sort_values('avg_daily_volume', ascending=False)
highest_liquidity.head(5)  # Top 10 most liquid with ≥10% ROI


,type_id,type_name,avg_lowest,avg_highest,avg_daily_volume,roi
11982,88087,Eleutrium,7.452581,9.249677,1.008197e+07,0.159877
51,212,Mjolnir Light Missile,25.105161,29.574516,2.080946e+06,0.100897
17,178,Carbonized Lead S,6.213871,7.527419,1.011630e+06,0.132077
670,2514,Inferno Rocket,4.591613,5.890323,6.709529e+05,0.198853
57,219,Thorium Charge S,12.789677,16.385161,6.320131e+05,0.197245
